In [1]:
import boto3
import data_collection
import data_storage
import json
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8')

bucket = "seatgeek-tickets"

In [5]:
success = False
while not success:
    try:
        data = data_collection.etl()
        success = True
    except Exception as e:
        print(f"Exiting data_collcetion ETL with the following exception: \n{e}")
resp = data_storage.put_object(data, bucket)

(1/17) https://seatgeek.com/nets-at-76ers-tickets/1-25-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775973
game=Brooklyn-Nets data-size=245
(2/17) https://seatgeek.com/nuggets-at-76ers-tickets/1-28-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775975
game=Denver-Nuggets data-size=245
(3/17) https://seatgeek.com/magic-at-76ers-tickets/1-30-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775977
game=Orlando-Magic data-size=240
(4/17) https://seatgeek.com/magic-at-76ers-tickets/2-1-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775979
game=Orlando-Magic data-size=245
(5/17) https://seatgeek.com/knicks-at-76ers-tickets/2-10-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775981
game=New-York-Knicks data-size=245
(6/17) https://seatgeek.com/rockets-at-76ers-tickets/2-13-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775983
game=Houston-Rockets data-size=245
(7/17) https://seatgeek.com/cavaliers-at-76ers-tickets/2-15-2023-philadelphia-pennsylv

In [ ]:
# master_df = pd.concat(dfs)
# master_df['section'] = master_df['section'].apply(lambda x: x.split(',')[0])

# section_size = master_df.groupby('section').size()
# valid_sections = section_size.index.values[section_size > 3]
# valid_sections